In [9]:
import pandas as pd
import random
import numpy as np
#import matplotlib.pyplot as plt
#import scipy.stats as stats
#import ipywidgets as widgets
import math
from IPython.display import display
from math import comb

Chance for a single die is mutually exclusive and exhaustive
D1 = dice is 1
D2 = dice is 2
D6 = dice is 6
P(D1) = P(D2) = P(D6) = 1/6
P(D1 v D2 v ... v D6) = 1
P(D1 & D2) = P(D2 & D6) = 0

Part 1: 
With 63 points in first round, we get a bonus of 50 points.
- We want to have an average of three of a kind in each instance because that will give us the bonus.
P(D1)
We should always try to get the bonus.

We also don't want the very lowest number to have too much of a say, so we penalize it

In [10]:
def inference_partOne(dice):
    
    p_outcome = 1/6
    p_not_outcome = 1-p_outcome
    
    #Dictionary of the choices and their given score at the end
    prob_dict = {}

    for d in set(dice):
        # How many of a kind
        count = dice.count(d)

        # Probability of a yahtzee our next turn.
        ytzy_prob = math.pow(p_outcome, 5-count)
        
        # probability that somethings gets 3 of a kind. Heavily benefits if its over, even though its still just 100% likely.
        beat3x_prob = math.pow(p_outcome,(3-count))
        
        # Overall score calculated on the probabilities and dice
        ovral_score = (ytzy_prob+beat3x_prob)
        
        # Reduce the weight of the overall score for number 1,
        # by 1/3
        if (d == 1):
            ovral_score = ovral_score*(1-1/3)
            
        # Round of because we dont care about super small variations in probability
        prob_dict[d] = round(ovral_score, 6)
    
    return prob_dict        

In [76]:
def probForDice(dice, similar):
    p_not_dice = (len(dice)-len(similar))/len(dice)
    print(p_not_dice)

    available = dice-(dice.count(similar)*len(similar))
    print(available)

    #P(at least one of similar)
    return (1-(math.pow(p_not_dice, available)))
    

In [77]:
d33 = [2,2,3,3,6]
s33 = [2,3]
print(probForDice(d33, s33))

0.6


TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [12]:
def keepMutipleInference(dice, similar):
#Dictionary of the choices and their given score at the end
    p_outcome = 1/6
    keep_dict = {}
    count = dice.count(similar[0])
    
    for i in range(len(similar)):
        index = i+1
        p_ytze = 0
        p_3x = 0
        dice_to_roll = 5-(count*index)
        
        #Its only possible with yahtzee if we keep 1 kind.
        if (index == 1):
            p_ytze = math.pow(p_outcome, 5-count)

        #Its not possible with three of a kind if we try to keep four vales.
        if(index <= 3):
            p_3x = probForDice(dice, similar)
        
        ovral = p_ytze+p_3x

        keep_dict[index] = round(ovral, 6)




In [13]:
def decision(dice, available, prob_dict):

    # If we didnt roll any available dice, then pick the highest one available.
    if not any(key in available for key in prob_dict.keys()):
        print("no like")
        return available[-1]
    
    # Find highest probabilities, can be multiple 
    high_key = max(prob_dict, key=prob_dict.get)
    same_value_keys = [key for key, value in prob_dict.items() if value == prob_dict[high_key]]
    if len(same_value_keys) > 1:
        print("Multiple ones...")
        print(same_value_keys)
    else:
        print(f"One high {high_key}")
        print(dice)

    # Should we pick multiple dice to keep


    return dice[0]

In [14]:
def keepDice(prob_dict, available):
    
    # If we didnt roll any available dice, then pick the highest one.
    if not any(key in available for key in prob_dict.keys()):
        return available[-1]

    #highest key
    high_key = 0
    #highest value
    high_val = 0
    
    for key, value in prob_dict.items():
        if (value >= high_val) and key in available:
            high_val = value
            high_key = key
    
    return high_key

In [15]:
def rollDice():
    return sorted([random.randint(1, 6) for _ in range(5)])

def rollDiceKeepType(dice, keep):
    copy = dice.copy()
    for i in range(len(dice)):
        if (dice[i] != keep):
            copy[i] = random.randint(1,6)
    
    return sorted(copy)


In [16]:
def oneGameSim(available):
    score = 0
    kept = 0
    dice = rollDice()
    print("intial roll")
    print(dice)
    #Test specific dice combination
    #dice = [1,1,1,4,4]
    turn = 2
    for i in range (turn):
        # Calculate probability for high scores
        prob_dict = inference_partOne(dice)
        # Get the one that is kept based on prob_dict
        kept = keepDice(prob_dict, available)
        dice = rollDiceKeepType(dice, kept)
        print("-.-.-.-.-.-")
        decision(dice, available, prob_dict)
        print("-.-.-.-.-.-")

    

    count = dice.count(kept)
    if (kept > 3 and count < 3 and available.count(1) != 0):
        # Use extra life we shouldt waste our good slots on bad numbers
        kept = 1

    count = dice.count(kept)
    score += count*kept

    
    return (score, kept)
    

In [67]:
repeat = 1
score = 0
available = [1, 2, 3, 4, 5, 6]
# Game Simulation
for _ in range(repeat):
   #while(len(available) > 0):
   s, k = oneGameSim(available)
   available.remove(k)
   score += s
   print(f"used: {k}")
   print(f"available: {available}")
   print(f"new score: {score}")
   print("---------------")
   print(f"Final score: {score}")



# Add logic for final selection
# [6,6,2,3,3] as a final roll will say we should pick 6, but then we use up that spot.


intial roll
[1, 1, 2, 4, 6]
-.-.-.-.-.-
One high 1
[1, 1, 1, 3, 4]
-.-.-.-.-.-
-.-.-.-.-.-
One high 1
[1, 1, 1, 2, 6]
-.-.-.-.-.-
used: 1
available: [2, 3, 4, 5, 6]
new score: 3
---------------
Final score: 3


None
